# Unsloth: Continued Pretraining (New Language)

This notebook demonstrates how to perform **Continued Pretraining** to teach an LLM a new language (e.g., Hindi) or a new domain.

Unlike instruction tuning, here we train on raw text data to improve the model's fundamental understanding of the new language.

In [ ]:
%%capture
!pip install unsloth
!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit", # Using a 1B model for speed
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### LoRA Configuration
We target all linear layers to maximize the model's ability to adapt to the new language structure.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Higher rank is better for learning new languages
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"], # Train embeddings and head too!
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

### Data Preparation
We will use a subset of the **Oscar** dataset for Hindi (or any target language). We format it as raw text.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("oscar", "unshuffled_deduplicated_hi", split = "train[:1%]") # 1% for demo

def formatting_prompts_func(examples):
    text = examples["text"]
    return { "text" : text, }
dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Packing is highly recommended for pretraining
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer.train()